In [ ]:
# !pip install transformers langchain torch faiss-cpu numpy
# !pip install sentence-transformers
# !pip install pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import os
from urllib.request import urlretrieve
import numpy as np
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
os.makedirs("Pakistan", exist_ok=True)
files = [
    "https://info.publicintelligence.net/MCIA-PakistanHandbook.pdf"
]
for url in files:
    file_path = os.path.join("Pakistan", url.rpartition("/")[2])
    urlretrieve(url, file_path)

In [ ]:
loader = PyPDFDirectoryLoader("./Pakistan/")

docs_before_split = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap  = 50,
)
docs_after_split = text_splitter.split_documents(docs_before_split)

docs_after_split[0]

Document(metadata={'source': 'Pakistan/MCIA-PakistanHandbook.pdf', 'page': 0}, page_content='Pakistan Country Handbook\nThis handbook provides basic reference information on Pakistan, including its \ngeography, history, government, military forces, and communications and trans-portation networks. This information is intended to familiarize military per\n s\nonnel \nwith local customs and area knowledge to assist them during their assignment to Pakistan. \nThe Marine Corps Intel\n l\nigence Activity is the community coordinator for the \nCountry Hand  b\nook Program. This product reflects the coordinated U.S. Defense \nIntelligence Community position on Pakistan .Dissemination and use of this publication is restricted to official military and')

In [ ]:
avg_doc_length = lambda docs: sum([len(doc.page_content) for doc in docs])//len(docs)
avg_char_before_split = avg_doc_length(docs_before_split)
avg_char_after_split = avg_doc_length(docs_after_split)

print(f'Before split, there were {len(docs_before_split)} documents loaded, with average characters equal to {avg_char_before_split}.')
print(f'After split, there were {len(docs_after_split)} documents (chunks), with average characters equal to {avg_char_after_split} (average chunk length).')

Before split, there were 296 documents loaded, with average characters equal to 995.
After split, there were 592 documents (chunks), with average characters equal to 508 (average chunk length).


In [ ]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

In [ ]:
sample_embedding = np.array(huggingface_embeddings.embed_query(docs_after_split[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 1.43675804e-02  2.05621347e-02 -5.27524343e-03 -4.33683023e-02
  3.53559405e-02 -2.75353193e-02  5.71042635e-02  1.29494378e-02
 -1.00698784e-01  8.40507261e-03  6.34408370e-02 -2.81770136e-02
  4.30057049e-02 -3.54474001e-02 -2.96685621e-02 -6.74374700e-02
  1.09389389e-03 -3.30156907e-02  2.20131185e-02 -5.80267012e-02
  1.70874391e-02  9.20885801e-03 -5.56742437e-02  8.73683989e-02
  7.60229230e-02  6.52979463e-02  3.42497043e-02  2.41889507e-02
 -5.60614616e-02 -1.51428938e-01 -9.08630248e-03  1.31402249e-02
 -2.60648075e-02  9.43885371e-03 -4.88951579e-02  2.94795670e-02
  3.01087815e-02  1.74482260e-02 -2.54820473e-02  8.53919890e-03
  2.75449753e-02  3.21992822e-02  1.20137252e-01  5.66879213e-02
  2.49452833e-02 -3.96076888e-02  1.22199440e-03  5.19754104e-02
  1.58972181e-02 -4.60061431e-02  2.00814810e-02  2.85943393e-02
  6.53216690e-02  2.12636758e-02 -1.71494354e-02 -2.31498387e-02
  1.17936703e-04  1.65618472e-02  4.09649387e-02  7

In [ ]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)


In [ ]:
query = """What is served after meal in Pakistan?"""
relevant_documents = vectorstore.similarity_search(query)
print(f'There are {len(relevant_documents)} documents retrieved which are relevant to the query. Display the first one:\n')
print(relevant_documents[0].page_content)

There are 4 documents retrieved which are relevant to the query. Display the first one:

Eating
al most all Pakistanis follow three Muslim dietary customs: they do not eat pork, they do not drink alcohol, and they fast during the month of Ramadan. During Ramadan, Muslims fast from sunup to sundown in order to learn discipline and self-control. 
i
slam does 
not require the elderly, pregnant women, or children who have not reached puberty to fast.
Pakistani cuisine is diverse with 
af
ghan- ir
anian or in
dian influ-
ences. Pakistanis sit on the floor and eat with the right hand. in 
P
akistani culture, the left hand is often used for unsanitary pur-
poses so it should not be raised to the mouth, dipped in a com-


In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:

from langchain_community.llms import HuggingFaceHub

HUGGINGFACEHUB_API_TOKEN = ">>>>>>>>>>>>>>>>>>>>>>"

hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
    model_kwargs={"temperature": 0.1, "max_length": 500}
)

query = """What is served after meal in Pakistan?"""
response = hf.invoke(query)
print(response)


What is served after meal in Pakistan?

- After the meal, the host will serve you tea or coffee. If you are invited to a Pakistani home, you will be served tea or coffee after the meal.

## What is the most popular food in Pakistan?

The 10 Most Popular Pakistani Foods

- Biryani. Biryani is a rice dish that is popular in Pakistan and India.
- Nihari. Nihari is a stew made with
